In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy.stats import mode
import pandas as pd
import seaborn as sns
from io import StringIO
import requests
import json
from sklearn.preprocessing import StandardScaler  # For scaling dataset
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation #For clustering
from sklearn.mixture import GaussianMixture
%matplotlib inline

In [2]:
tmp = pd.read_csv('Beehive Metrics Dataset/temperature_2017.csv')
tmp.head()

,timestamp,temperature
0,2017-01-01 05:00:00,-1.911244
1,2017-01-01 06:00:00,-1.866717
2,2017-01-01 07:00:00,-2.077833
3,2017-01-01 08:00:00,-2.327167
4,2017-01-01 09:00:00,-2.791233


In [3]:
hum = pd.read_csv('Beehive Metrics Dataset/humidity_2017.csv')
hum.head()

,timestamp,humidity
0,2017-01-01 05:00:00,92.406667
1,2017-01-01 06:00:00,92.270000
2,2017-01-01 07:00:00,92.575000
3,2017-01-01 08:00:00,92.840000
4,2017-01-01 09:00:00,93.640000


In [4]:
#Change the timestamp fields to the datetime data type
tmp['timestamp'] = pd.to_datetime(tmp['timestamp'])
hum['timestamp'] = pd.to_datetime(hum['timestamp'])

In [5]:
#Check the datatypes
tmp.info()
hum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401869 entries, 0 to 401868
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   timestamp    401869 non-null  datetime64[ns]
 1   temperature  401866 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8737 entries, 0 to 8736
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  8737 non-null   datetime64[ns]
 1   humidity   8737 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 136.6 KB


In [6]:
#Count blanks or whitespaces
tmp = tmp.replace(r'\s+\t+', np.nan, regex=True).replace('', np.nan)
#Is anything null?
tmp.isnull().sum()

timestamp      0
temperature    3
dtype: int64

In [7]:
hum = hum.replace(r'\s+\t+', np.nan, regex=True).replace('', np.nan)
hum.isnull().sum()

timestamp    0
humidity     0
dtype: int64

In [8]:
#Remove what we need to from tmp
tmp = tmp.drop([163499])
#Remove what we need to from hum
hum = hum.drop([3006, 3007])
#Drop nulls as well
tmp = tmp.dropna()

In [9]:
tmp.isnull().sum()

timestamp      0
temperature    0
dtype: int64

timestamp      0
humidity       0
temperature    0
dtype: int64

In [13]:
#Merge the two together,this is going to give us a bunch of nulls since the temp readings become more frequent
df = hum.merge(tmp, how='left', on='timestamp')
df.head(20)

,timestamp,humidity,temperature
0,2017-01-01 05:00:00,92.406667,-1.911244
1,2017-01-01 05:00:00,92.406667,-0.268978
2,2017-01-01 05:00:00,92.406667,0.553844
3,2017-01-01 05:00:00,92.406667,3.427489
4,2017-01-01 05:00:00,92.406667,2.186400
5,2017-01-01 05:00:00,92.406667,2.848156
6,2017-01-01 05:00:00,92.406667,4.505489
7,2017-01-01 05:00:00,92.406667,1.364311
8,2017-01-01 05:00:00,92.406667,3.199022
9,2017-01-01 05:00:00,92.406667,1.090467
